In [3]:
import pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import math

In [4]:
def get_outlier_indices(df, features):
    outlier_indices = []

    for c in features:
        Q1 = df[c].quantile(0.25)
        Q3 = df[c].quantile(0.75)
        IQR = Q3 - Q1

        outlier_step = 1.5 * IQR

        outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)

    outlier_indices = list(set(outlier_indices))
    return outlier_indices

def remove_outliers(df, features):
    outlier_indices = get_outlier_indices(df, features)
    df_cleaned = df.drop(outlier_indices)
    df_cleaned.reset_index(drop=True, inplace=True)
    return df_cleaned

In [5]:
df = pandas.read_csv('Clean_Dataset.csv')
columns_to_drop = ['Unnamed: 0', 'flight']
df = df.drop(columns=columns_to_drop)
le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])

# One-hot encoding
columns_to_onehot = ['airline', 'source_city', 'destination_city', 'departure_time', 'arrival_time', 'stops']

ohe = OneHotEncoder(drop='first', sparse_output=False)
one_hot_encoded = ohe.fit_transform(df[columns_to_onehot])

df_encoded = pandas.DataFrame(one_hot_encoded, columns=ohe.get_feature_names_out(columns_to_onehot))
df = pandas.concat([df.drop(columns=columns_to_onehot), df_encoded], axis=1)
df = remove_outliers(df, ['duration', 'price'])
X = df.drop(columns='price')
y = df.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

## Linear regression

In [6]:
lr = LinearRegression().fit(X, y)
y_pred_lr = lr.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_pred_lr)
rmse = math.sqrt(mse)
mae = metrics.mean_absolute_error(y_test, y_pred_lr)
print("MSE: " + str(mse))
print("RMSE: " + str(rmse))
print("MAE: " + str(mae))

MSE: 44809694.61106912
RMSE: 6694.004377879441
MAE: 4558.017302847961


## Random forest

In [5]:
rf = RandomForestRegressor(n_estimators=500, max_features=8, n_jobs=-1).fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_pred_rf)
rmse = math.sqrt(mse)
mae = metrics.mean_absolute_error(y_test, y_pred_rf)
print("MSE: " + str(mse))
print("RMSE: " + str(rmse))
print("MAE: " + str(mae))

MSE: 7599464.922266816
RMSE: 2756.7127021629976
MAE: 1158.2103852486134


## Multilayer Perceptron

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import models, layers
from sklearn.metrics import mean_squared_error
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print(f'Test MAE: {test_mae}')

y_pred = model.predict(X_test_scaled)

# Calculate and print MSE and RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Test MSE: {mse}')
print(f'Test RMSE: {rmse}')

Epoch 1/10
5959/5959 [==============================] - 19s 3ms/step - loss: 124342488.0000 - mae: 6223.3711 - val_loss: 29672728.0000 - val_mae: 3349.4348
Epoch 2/10
5959/5959 [==============================] - 18s 3ms/step - loss: 27883768.0000 - mae: 3232.3948 - val_loss: 28103098.0000 - val_mae: 3196.2537
Epoch 3/10
5959/5959 [==============================] - 18s 3ms/step - loss: 26682114.0000 - mae: 3126.9985 - val_loss: 27086154.0000 - val_mae: 3145.5420
Epoch 4/10
5959/5959 [==============================] - 18s 3ms/step - loss: 25881238.0000 - mae: 3066.3037 - val_loss: 26178814.0000 - val_mae: 3067.0283
Epoch 5/10
5959/5959 [==============================] - 18s 3ms/step - loss: 25068414.0000 - mae: 3005.2976 - val_loss: 25378264.0000 - val_mae: 3024.3074
Epoch 6/10
5959/5959 [==============================] - 18s 3ms/step - loss: 24157248.0000 - mae: 2940.4316 - val_loss: 24228226.0000 - val_mae: 2924.5330
Epoch 7/10
5959/5959 [==============================] - 18s 3ms/step 

## Bagging Regression Tree

In [10]:
base_regressor = DecisionTreeRegressor(random_state=42)

# Create a Bagging Regressor with Decision Trees as base estimators
bagging_regressor = BaggingRegressor(base_regressor, n_estimators=10, random_state=42)

# Define the parameter grid for hyperparameter tuning
# param_grid = {
#     'n_estimators': [5, 10, 15],
#     'max_samples': [0.5, 0.7, 1.0],
#     'max_features': [0.5, 0.7, 1.0]
# }

# # Use GridSearchCV for hyperparameter tuning
# grid_search = GridSearchCV(bagging_regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train, y_train)

# Fit the Bagging Regressor on the training data
bagging_regressor.fit(X_train, y_train)

# Make predictions on the test data
y_pred = bagging_regressor.predict(X_test)

mse = metrics.mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
mae = metrics.mean_absolute_error(y_test, y_pred)
print("MSE: " + str(mse))
print("RMSE: " + str(rmse))
print("MAE: " + str(mae))

MSE: 7917499.504909788
RMSE: 2813.8051647030907
MAE: 1095.8675191186674
